In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
from local_enc import *
from global_oneshot import *
## if reparameterize continuous variables
Reparameterized = False
from os_ep import *
## Load dataset
data_path = "../gmm_dataset_3c"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 100
NUM_HIDDEN_LOCAL = 32

BATCH_SIZE = 20
NUM_EPOCHS = 100
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()

DEVICE = torch.device('cuda:1')
Model_Params = (N, K, D)

NUM_RUNS = 5

In [ ]:
for i in range(NUM_RUNS):
    print('run : %d starting' % i)
    path = 'baseline-mlp-100samples-%dround' % i
    Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, path)
    enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
    oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
    if CUDA:
        enc_z.cuda().to(DEVICE)
        oneshot_eta.cuda().to(DEVICE)
    optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
    models = (oneshot_eta, enc_z)
    train_baseline(models, EP, optimizer, Data, Model_Params, Train_Params)
    torch.save(enc_z.state_dict(), '../weights/enc-z-%s' % path)
    torch.save(oneshot_eta.state_dict(), '../weights/oneshot-eta-%s' % path)
    

run : 0 starting


In [ ]:
def test(models, objective, Data, Model_Params, Train_Params):
    (NUM_EPOCHS, NUM_DATASETS, S, B, CUDA, device, path) = Train_Params
    SubTrain_Params = (device, S, B) + Model_Params
    ##(N, K, D, mcmc_size) = Model_Params
#     indices = torch.randperm(NUM_DATASETS)
    step = 8
    indices = torch.arange(NUM_DATASETS)
    batch_indices = indices[step*B : (step+1)*B]
    print(batch_indices)
    obs = Data[batch_indices]
    obs = shuffler(obs).repeat(S, 1, 1, 1)
    if CUDA:
        obs =obs.cuda().to(device)
    loss, metric_step, reused = objective(models, obs, SubTrain_Params)
    return obs, metric_step, reused

In [ ]:
BATCH_SIZE_TEST = 25
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, 1, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
obs, metric_step, reused = test(models, EP, Data, Model_Params, Train_Params_Test)
(q_eta, _, q_z, _, _, _) = reused

In [ ]:
def plot_samples(obs, q_eta, q_z, PATH):
    colors = ['#0077BB', '#009988', '#EE7733', 'm']
    fig = plt.figure(figsize=(25,25))
    xs = obs[0].cpu().data.numpy()
#     E_z = q_z['zs'].value.cpu().data.numpy()[0]
#     E_mu = q_eta['means'].value.cpu().data.numpy()[0]
#     E_tau = q_eta['precisions'].value.cpu().data.numpy()[0]
    E_z = q_z['zs'].dist.probs[0].cpu().data.numpy()
    batch_size, N, K = E_z.shape
    _, _, D = xs.shape
    E_mu = q_eta['means'].dist.loc[0].cpu().data.numpy()
    E_tau = (q_eta['precisions'].dist.concentration[0] / q_eta['precisions'].dist.rate[0]).cpu().data.numpy()
    for b in range(batch_size):
        ax = fig.add_subplot(int(batch_size / 5), 5, b+1)
        xb = xs[b]
        zb = E_z[b]
        mu = E_mu[b].reshape(K, D)
        sigma2 = 1. / E_tau[b]
        assignments = zb.argmax(-1)
        for k in range(K):
            cov_k = np.diag(sigma2[k])
            xk = xb[np.where(assignments == k)]
            ax.scatter(xk[:, 0], xk[:, 1], c=colors[k])
            plot_cov_ellipse(cov=cov_k, pos=mu[k], nstd=2, ax=ax, alpha=0.2, color=colors[k])
        ax.set_ylim([-15, 15])
        ax.set_xlim([-15, 15])
    plt.savefig('../results/modes-' + PATH + '.svg')
    
    
%time plot_samples(obs, q_eta, q_z, 'test')

In [ ]:
list(oneshot_eta.parameters())

In [ ]:
np.array([[-0.4972, -1.0698], [ 0.7994,  0.6396], [-1.0111,  0.8875]])

In [ ]:
aa = obs[0,0].cpu().data.numpy()
plt.scatter(aa[:,0], aa[:,1])
plt.xlim([-15,15])
plt.ylim([-15,15])

In [ ]:
indices = torch.arange(NUM_DATASETS)
step = 2
batch_indices = indices[step*BATCH_SIZE_TEST : (step+1)*BATCH_SIZE_TEST]
bb = Data[batch_indices]
bb = bb[0].data.numpy()

plt.scatter(bb[:,0], bb[:,1])
plt.xlim([-15,15])
plt.ylim([-15,15])

In [ ]:
Data[]